This notebook finds that neuron-ids in ground-truth that have the worst false merges when we run inference. The following uses the inference of modelA on subvolume [0,0,0] - [125, 128, 128] of cremiA.

In [1]:
from ffn.inference import storage
import h5py
import numpy as np

from cremi_python.cremi.evaluation.rand import adapted_rand
from cremi_python.cremi.evaluation.voi import voi, sorted_vi_components

# Load segmentation and ground-truth

In [2]:
#load segmentation, https://drive.google.com/drive/folders/1SDjXXM6xW6sjzEJeUrDduL_PTQTI1Skz?usp=sharing
seg, prob = storage.load_segmentation('0/0', (0, 0, 0))
print(seg.shape)

# load ground-truth, https://drive.google.com/file/d/1soEqM2DPVT6hYUPLybvNGIjtTYJbdrGl/view?usp=sharing
filename = './labeledA.hdf'
with h5py.File(filename, 'r') as f:
    raw = f['volumes/raw'][...]
    truth = f['volumes/labels/neuron_ids'][...]
raw = raw[:125, :128, :128]
truth = truth[:125, :128, :128]

(125, 128, 128)


In [3]:
raw

array([[[118, 111, 103, ..., 167, 158, 161],
        [112, 102, 102, ..., 160, 159, 160],
        [110, 105, 105, ..., 160, 161, 148],
        ...,
        [144, 137, 140, ..., 161, 159, 155],
        [126, 125, 125, ..., 157, 157, 159],
        [110, 110, 114, ..., 160, 154, 151]],

       [[134, 138, 131, ..., 113, 112, 119],
        [140, 142, 137, ..., 115, 113, 118],
        [143, 140, 136, ..., 113, 112, 115],
        ...,
        [148, 158, 160, ..., 161, 159, 160],
        [151, 159, 155, ..., 162, 159, 162],
        [150, 155, 158, ..., 154, 160, 162]],

       [[ 93,  86,  82, ..., 111, 114, 117],
        [ 86,  82,  80, ..., 112, 125, 131],
        [ 71,  72,  68, ..., 122, 132, 140],
        ...,
        [122, 122, 123, ..., 141, 136, 132],
        [128, 131, 134, ..., 154, 148, 143],
        [142, 138, 149, ..., 158, 155, 153]],

       ...,

       [[124, 128, 124, ..., 139, 136, 135],
        [115, 118, 121, ..., 147, 139, 142],
        [109, 108, 110, ..., 144, 146, 137

In [4]:
# Calculate rand and voi score

In [5]:
adapted_rand(seg, truth, True)

(0.2504528800746295, 0.9835280998897968, 0.6054991020899303)

In [6]:
voi(seg, truth)

(1.5582847410016272, 0.6238707278010233)

# Find worst false merges

In [7]:
false_merges, entropy, _, _ = sorted_vi_components(seg, truth, compress=False)


In [8]:
ids = false_merges[entropy>0]

In [9]:
# These are the gt (ground-truth) ids that have the worst false merges. 
ids

array([  5556,   2064,   7550,   2193,   3538,   6252,   3040,   9080,
         9284,  53070, 188748, 175398,   3877,   9988, 123707,  70515,
       123694,  37473,  21635, 180064, 180429,  31621, 114415,   7113,
       101706,  35891, 181339, 177085, 183060, 180885,  53346, 135269,
        68201, 132030,   5918,  39852, 180903, 179183,  55330, 181345,
       135267, 189390, 182193, 183494, 187328, 187825,  71424,  87656,
       178398, 175340, 166017, 185566, 188674, 189385, 165252, 179174,
       131650,    885, 166452,  47677, 135261, 185108, 150759, 133730,
       180443,  78629, 169519, 189389, 144909, 178741, 131647, 177890,
       169984,  62462, 186379, 146056, 189386, 183879, 131114, 115690,
       135262, 189301, 145660, 162742, 144541, 131643, 131667, 174982,
       184703, 174558, 131651, 144922, 189381, 180084, 115586, 189387,
       179607, 188675,  10561, 167947, 125518, 162781, 131639, 139395,
       167248, 135271, 176666, 131670, 133733, 172545, 186010, 172926,
      

In [10]:
len(set(ids))

262

In [11]:
len(np.unique(truth))

588

# Next step: visualize

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
# # get a sorted list of ground truth neuron ids sorted in descending order
# # by size/number of voxels
# truth_unique, truth_unique_counts = np.unique(truth, return_counts=True)
# _p = (-truth_unique_counts).argsort()
# sorted_truth_unique = truth_unique[_p]
# 
# # get mask for one specific neuron, call it raw2
# nth_most_common_truth_id = 2
# voxels_one_id = np.where(truth==sorted_truth_unique[nth_most_common_truth_id])
# raw2 = np.zeros((125, 128, 128))
# raw2[voxels_one_id] = 1
# 
# # now we want to get a list of inference ids that take up that space
# seg_ids_in_mask, seg_ids_in_mask_counts = np.unique(seg[voxels_one_id], return_counts=True)
# _p = (-seg_ids_in_mask_counts).argsort()
# # just focus on largest seg object for now i.e. [:1]
# sorted_seg_ids_in_mask = seg_ids_in_mask[_p][:1]
# 
# # plot each segmentation id in a differnt color
# seg_plot = np.zeros((125, 128, 128))
# seg_plot_colors = np.linspace(1, .5, len(sorted_seg_ids_in_mask))
# for i in range(len(sorted_seg_ids_in_mask)):
#     seg_plot[np.where(seg==sorted_seg_ids_in_mask[i])] = seg_plot_colors[i]
#     
# layer = 3
# 
# fig, axarr = plt.subplots(1,3,figsize=(20,20))
# 
# axarr[0].imshow(raw[layer])
# axarr[1].imshow(raw2[layer])
# axarr[2].imshow(seg_plot[layer])
# 

In [33]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

import ipywidgets as ipyw

class CREMIInferenceViewer:
    def __init__(self, raw, truth, seg, top_n_gt=10, figsize=(20,20)):
        self.raw = raw
        self.truth = truth
        self.seg = seg
        self.top_n_gt = top_n_gt # plot only n largest neurons in ground truth set
        self.figsize = figsize
        
        # get a sorted list of ground truth neuron ids sorted in descending order
        # by size/number of voxels
        self.truth_unique, self.truth_unique_counts = np.unique(self.truth, return_counts=True)
        self._p = (-self.truth_unique_counts).argsort()
        self.sorted_truth_unique = self.truth_unique[self._p]
        self.len_sorted_truth_unique = len(self.sorted_truth_unique)
        
        """Call to select neuron_id, only allow top 10 neuron_ids"""
        ipyw.interact(self.neuron_selection, neuron=ipyw.SelectionSlider(
            options=self.sorted_truth_unique[:self.top_n_gt],
            value=self.sorted_truth_unique[0],
            description='Neuron_id:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True))
        
    def neuron_selection(self, neuron):
        """Transform neuron_ids such that selected neuron is 1 else 0"""
        # get mask for one specific neuron, call it raw2
        self.nth_most_common_truth_id = neuron
        self.voxels_one_id = np.where(self.truth==neuron)
        self.raw2 = np.zeros((125, 128, 128))
        self.raw2[self.voxels_one_id] = 1

        # now we want to get a list of inference ids that take up that space
        self.seg_ids_in_mask, self.seg_ids_in_mask_counts = np.unique(self.seg[self.voxels_one_id], return_counts=True)
        self._p = (-self.seg_ids_in_mask_counts).argsort()
        # just focus on largest seg object for now i.e. [:1]
        self.sorted_seg_ids_in_mask = self.seg_ids_in_mask[self._p][:1]

        # plot each segmentation id in a differnt color
        self.seg_plot = np.zeros((125, 128, 128))
        self.seg_plot_colors = np.linspace(1, .5, len(self.sorted_seg_ids_in_mask))
        for i in range(len(self.sorted_seg_ids_in_mask)):
            self.seg_plot[np.where(self.seg==self.sorted_seg_ids_in_mask[i])] = self.seg_plot_colors[i]
        
        """Call to view a slice within the selected slice plane"""
        self.maxZ = self.raw.shape[2] - 1
        ipyw.interact(self.plot_slice, z=ipyw.IntSlider(
            min=0, 
            max=self.maxZ, 
            step=1, 
            continuous_update=False, 
            description='Image Slice:'))                  
            
    def plot_slice(self, z):
        
        self.fig, self.axarr = plt.subplots(1,3,figsize=(20,20))
        self.axarr[0].imshow(self.raw[z], cmap='gray')
        self.axarr[0].title.set_text('Raw image')
        self.axarr[1].imshow(self.raw2[z])
        self.axarr[1].title.set_text('Ground truth neuron')
        self.axarr[2].imshow(self.seg_plot[z])
        self.axarr[2].title.set_text('Inference result')
        
        plt.show()
        


In [34]:
CREMIInferenceViewer(raw, truth, seg, top_n_gt=10, figsize=(20,20))

interactive(children=(SelectionSlider(continuous_update=False, description='Neuron_id:', options=(5556, 2064, …